## ENGRI 1120 Mole Balance Flux Balance Analysis Example

### Introduction

Suppose we have an open system with species $\mathcal{M}$, streams $\mathcal{S}$ and reactions $\mathcal{R}$. Further, suppose we partition the stream set $\mathcal{S}$ into streams entering the system $\mathcal{S}^{+}$, and streams leaving the system $\mathcal{S}^{-}$. Then, the steady-state species mole balances are given by:

$$\sum_{s\in\mathcal{S}^{+}}\dot{n}_{si} - \sum_{k\in\mathcal{S}^{-}}\dot{n}_{ki} + \sum_{j\in\mathcal{R}}\sigma_{ij}\dot{\epsilon}_{j} = 0\qquad\forall{i}\in\mathcal{M}$$

Finally, we know that $\dot{n}_{i,j}\geq{0}$ for every $i$ and $j$; species mole flows must be non-negative. Then, the (unknown) open extents $\dot{\epsilon}_{j}$ are the solution of a linear programming problem in which the linear objective $\mathcal{O}$:

$$\text{maximize/minimize}~\mathcal{O} = \sum_{j\in\mathcal{R}}c_{j}\dot{\epsilon}_{j}$$

is minimized (or maximized) subject to a collection linear and bounds constraints:

$$\begin{eqnarray}
\sum_{j\in\mathcal{R}}\sigma_{ij}\dot{\epsilon}_{j}&\geq&{-\sum_{s\in\mathcal{S}^{+}}\dot{n}_{si}}\qquad\forall{i}\in\mathcal{M}\\
\mathcal{L}_{j}&\leq\dot{\epsilon}_{j}\leq&\mathcal{U}_{j}\qquad\forall{j}\in\mathcal{R}
\end{eqnarray}$$

### Problem
Fill me in

### Example setup

In [1]:
import Pkg; Pkg.activate("."); Pkg.resolve(); Pkg.instantiate();

  Activating project at `~/Desktop/julia_work/ENGRI-1120-IntroToChemE-Example-Notebooks/notebooks-jupyter/ENGRI-1120-Toy-FBA-Example`
  No Changes to `~/Desktop/julia_work/ENGRI-1120-IntroToChemE-Example-Notebooks/notebooks-jupyter/ENGRI-1120-Toy-FBA-Example/Project.toml`
  No Changes to `~/Desktop/julia_work/ENGRI-1120-IntroToChemE-Example-Notebooks/notebooks-jupyter/ENGRI-1120-Toy-FBA-Example/Manifest.toml`


In [2]:
# load reqd packages and set paths -
using PrettyTables
using GLPK

# setup paths -
const _ROOT = pwd();

In [3]:
include("ENGRI-1120-Example-CodeLib.jl");

#### a) Build the stoichiometric matrix $S$

<img src="figs/Fig-expa-ToyNetwork_v2.png" style="width:50%">

In [4]:
# Setup a collection of reaction strings -
reaction_array = Array{String,1}()

# encode the reactions -
# internal reactions -
push!(reaction_array,"v₁,A,B,false")
push!(reaction_array,"v₂,B,C,true")
push!(reaction_array,"v₃,C,D,true")
push!(reaction_array,"v₄,C,E,false")

# exchange reactions -
push!(reaction_array,"b₁,Ax,A,true")
push!(reaction_array,"b₂,Bx,B,true")
push!(reaction_array,"b₃,D,Dx,true")
push!(reaction_array,"b₄,E,Ex,true")

# compute the stoichiometric matrix -
# the optional expand arguement = should we split reversible reactions? (default: false)
(S, species_name_array, reaction_name_array) = build_stoichiometric_matrix(reaction_array; 
    expand=false);

In [5]:
(ℳ, ℛ) = size(S);

In [6]:
[1:ℳ species_name_array]

9×2 Matrix{Any}:
 1  "A"
 2  "Ax"
 3  "B"
 4  "Bx"
 5  "C"
 6  "D"
 7  "Dx"
 8  "E"
 9  "Ex"

In [7]:
[1:ℛ reaction_name_array]

8×2 Matrix{Any}:
 1  "v₁"
 2  "v₂"
 3  "v₃"
 4  "v₄"
 5  "b₁"
 6  "b₂"
 7  "b₃"
 8  "b₄"

#### b) Build the reaction bounds array

In [8]:
# setup the bounds array -
flux_bounds_array = zeros(ℛ,2);
flux_bounds_array[:,2] .= 100.0; # set a default value for the *upper* bound on the flux

# set an upper bound on v₂ -
# flux_bounds_array[2,2] = 1.0;

# show flux bounds table -
flux_bounds_table_data = Array{Any,2}(undef, ℛ,3);
for i ∈ 1:ℛ
    flux_bounds_table_data[i,1] = reaction_name_array[i];
    flux_bounds_table_data[i,2] = flux_bounds_array[i,1];
    flux_bounds_table_data[i,3] = flux_bounds_array[i,2];
end

# flux bounds header -
flux_bounds_header = (["Reaction", "Lᵢ", "Uᵢ"], ["", "mol/time", "mol/time"]);

# show bounds table -
pretty_table(flux_bounds_table_data; header = flux_bounds_header) 

┌──────────┬──────────┬──────────┐
│ Reaction │       Lᵢ │       Uᵢ │
│          │ mol/time │ mol/time │
├──────────┼──────────┼──────────┤
│       v₁ │      0.0 │    100.0 │
│       v₂ │      0.0 │    100.0 │
│       v₃ │      0.0 │    100.0 │
│       v₄ │      0.0 │    100.0 │
│       b₁ │      0.0 │    100.0 │
│       b₂ │      0.0 │    100.0 │
│       b₃ │      0.0 │    100.0 │
│       b₄ │      0.0 │    100.0 │
└──────────┴──────────┴──────────┘


#### c) Build the species bounds array

In [9]:
# setup the species bounds array -

# we know from out theory, that that the lower bound is -1*sum of the inputs 
ṅ₁ = zeros(ℳ);
ṅ₂ = zeros(ℳ);

# suppose we supply Ax in stream 1, and Bx in stream 2
ṅ₁[2] = 20.0; # supply Ax -
ṅ₂[4] = 5.0;  # supply Bx -

# setup -
species_bounds_array = [-1*(ṅ₁ .+ ṅ₂) 1000.0*ones(ℳ)];

# show species bounds table -
species_bounds_table_data = Array{Any,2}(undef, ℳ, 3);
for i ∈ 1:ℳ
    species_bounds_table_data[i,1] = species_name_array[i];
    species_bounds_table_data[i,2] = species_bounds_array[i,1];
    species_bounds_table_data[i,3] = species_bounds_array[i,2];
end

# flux bounds header -
species_bounds_header = (["Species i", "Lᵢ", "Uᵢ"], ["", "mol/time", "mol/time"]);

# show bounds table -
pretty_table(species_bounds_table_data; header = species_bounds_header) 

┌───────────┬──────────┬──────────┐
│ Species i │       Lᵢ │       Uᵢ │
│           │ mol/time │ mol/time │
├───────────┼──────────┼──────────┤
│         A │     -0.0 │   1000.0 │
│        Ax │    -20.0 │   1000.0 │
│         B │     -0.0 │   1000.0 │
│        Bx │     -5.0 │   1000.0 │
│         C │     -0.0 │   1000.0 │
│         D │     -0.0 │   1000.0 │
│        Dx │     -0.0 │   1000.0 │
│         E │     -0.0 │   1000.0 │
│        Ex │     -0.0 │   1000.0 │
└───────────┴──────────┴──────────┘


#### d) Set the objective coefficient vector

In [10]:
# setup the objective vector -
c = zeros(ℛ);
c[7] = -1.0;

#### e) Estimate the extent through the network

In [15]:
results = compute_optimal_extent(S, flux_bounds_array, species_bounds_array, c);

# check:
println("Exit flag: $(results.exit_flag) and status flag: $(results.status_flag)")

Exit flag: 0 and status flag: 5


In [16]:
# get the reaction extent vector -
ϵ̇ = results.calculated_flux_array

# build a table -
optimal_extent_table_data = Array{Any,2}(undef, ℛ, 2);
for i ∈ 1:ℛ
    optimal_extent_table_data[i,1] = reaction_name_array[i]
    optimal_extent_table_data[i,2] = ϵ̇[i]
end

# build header -
optimal_table_header = (["Reaction", "ϵ̇ᵢ"], ["", "mol/time"])

# show table -
pretty_table(optimal_extent_table_data; header=optimal_table_header)

┌──────────┬──────────┐
│ Reaction │       ϵ̇ᵢ │
│          │ mol/time │
├──────────┼──────────┤
│       v₁ │     20.0 │
│       v₂ │     25.0 │
│       v₃ │     25.0 │
│       v₄ │      0.0 │
│       b₁ │     20.0 │
│       b₂ │      5.0 │
│       b₃ │     25.0 │
│       b₄ │      0.0 │
└──────────┴──────────┘


In [17]:
# compute the output compostion -
ṅ₃ = ṅ₂ + ṅ₁ + S*ϵ̇;

# compute the change because of the reaction -
Δ = S*ϵ̇;


# build flow table -
flow_table_data = Array{Any,2}(undef, ℳ, 5);
for i ∈ 1:ℳ
    flow_table_data[i,1] = species_name_array[i]
    flow_table_data[i,2] = ṅ₁[i]
    flow_table_data[i,3] = ṅ₂[i]
    flow_table_data[i,4] = ṅ₃[i]
    flow_table_data[i,5] = Δ[i]
end

# setup header -
flow_header_data = (["Species", "ṅ₁", "ṅ₂","ṅ₃", "Δ"], ["", "mol/time", "mol/time", "mol/time", "mol/time"]) 

# show -
pretty_table(flow_table_data; header = flow_header_data)

┌─────────┬──────────┬──────────┬──────────┬──────────┐
│ Species │       ṅ₁ │       ṅ₂ │       ṅ₃ │        Δ │
│         │ mol/time │ mol/time │ mol/time │ mol/time │
├─────────┼──────────┼──────────┼──────────┼──────────┤
│       A │      0.0 │      0.0 │      0.0 │      0.0 │
│      Ax │     20.0 │      0.0 │      0.0 │    -20.0 │
│       B │      0.0 │      0.0 │      0.0 │      0.0 │
│      Bx │      0.0 │      5.0 │      0.0 │     -5.0 │
│       C │      0.0 │      0.0 │      0.0 │      0.0 │
│       D │      0.0 │      0.0 │      0.0 │      0.0 │
│      Dx │      0.0 │      0.0 │     25.0 │     25.0 │
│       E │      0.0 │      0.0 │      0.0 │      0.0 │
│      Ex │      0.0 │      0.0 │      0.0 │      0.0 │
└─────────┴──────────┴──────────┴──────────┴──────────┘
